In [242]:
import tensorflow as tf

import pandas as pd

import json
import os
from datetime import date,datetime,timedelta,timezone

from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest

print(tf.__version__)

2.11.0


In [243]:
#import functions_framework
# @functions_framework.http
# def predict_incident_severity_by_tf(request):

In [244]:
PATH_FOLDER_ARTIFACTS="model" 
#PATH_FOLDER_ARTIFACTS="tuned_model" 
#PATH_FOLDER_ARTIFACTS="gs://tf1-incident-smart-ml-yip/model"


showEvaluationReport=True

#model_version='v2_to_15072023'
model_version='v1_to_Mar23'

PROJECT_ID='smart-data-ml'
dataset_id='SMartML'

# client = bigquery.Client(PROJECT_ID)
credentials = service_account.Credentials.from_service_account_file(r'C:\Windows\smart-data-ml-91b6f6204773.json')
client = bigquery.Client(credentials=credentials, project=PROJECT_ID)


predict_from_date=os.environ.get('predict_from_date', '')
all_prediction=os.environ.get('all_prediction', '1')  # 1 is all , 0 is 1 day

print(f"Prediction From = {predict_from_date}")
print(f"All prediction = {all_prediction}")

# map_sevirity_to_class={'Cosmatic': 0, 'Minor': 1, 'Major': 2, 'Critical': 3}

Prediction From = 
All prediction = 1


In [245]:
table_id = f"{PROJECT_ID}.{dataset_id}.new_incident"
predictResult_table_id=f"{PROJECT_ID}.{dataset_id}.new_result_prediction_incident"
unUsedColtoPredict=['severity','id','severity_id','severity_name','imported_at']

In [246]:
print("Load label target multiclasses")
mapping_file="incident_severity_to_class.json"
with open(mapping_file, 'r') as json_file:
     map_sevirity_to_class= json.load(json_file)

print(map_sevirity_to_class)

Load label target multiclasses
{'Cosmetic': 0, 'Minor': 1, 'Major': 2, 'Critical': 3}


In [247]:
# Get today's date
prediction_datetime=datetime.now(timezone.utc)
today_str=prediction_datetime.strftime("%Y-%m-%d")
today=datetime.strptime(today_str,"%Y-%m-%d")
print(prediction_datetime)

2023-07-20 10:24:22.185365+00:00


In [248]:
# Yesterday date
if predict_from_date=='':
 yesterday = today - timedelta(days = 1)
 str_yesterday=yesterday.strftime('%Y-%m-%d')
else:
 str_yesterday=predict_from_date

str_today=today.strftime('%Y-%m-%d')

print(f"Get data between {str_yesterday} to {str_today} to predict sevirity level")

Get data between 2023-07-19 to 2023-07-20 to predict sevirity level


In [249]:
def load_data_bq(sql:str):
 
 query_result=client.query(sql)
 df=query_result.to_dataframe()
 return df

In [250]:
if int(all_prediction)==0:
    sql=f"""
    SELECT *  FROM `{table_id}` 
     WHERE DATE(imported_at) >= '{str_yesterday}' and DATE(imported_at) < '{str_today}' 
     order by imported_at
    """
else:
    sql=f"""
    SELECT *  FROM `{table_id}` 
     order by imported_at
    """

print(sql)


    SELECT *  FROM `smart-data-ml.SMartML.new_incident` 
     order by imported_at
    


In [251]:

#LIMIT 2
dfNewData=load_data_bq(sql)
dfNewData=dfNewData.drop_duplicates(subset=['id'],keep='last')

dfNewData.insert(2, 'severity', dfNewData['severity_name'].map(map_sevirity_to_class),True)


print(dfNewData.info())
# print(dfNewData)

if len(dfNewData)==0:
    print("No Data To predict")
    quit()
    #return "No Data To predict"
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         37 non-null     Int64         
 1   severity_id                37 non-null     Int64         
 2   severity                   37 non-null     int64         
 3   severity_name              37 non-null     object        
 4   sla                        37 non-null     object        
 5   product_type               37 non-null     object        
 6   brand                      37 non-null     object        
 7   service_type               37 non-null     object        
 8   incident_type              37 non-null     object        
 9   open_to_close_hour         37 non-null     float64       
 10  response_to_resolved_hour  37 non-null     float64       
 11  imported_at                37 non-null     datetime64[ns]
dtypes: Int64(2

In [252]:
try:
    model = tf.keras.models.load_model(PATH_FOLDER_ARTIFACTS)    
    print(f"Load from {PATH_FOLDER_ARTIFACTS}")
    # print(model.summary())
except Exception as error:
    
  print(str(error))
  raise error


Load from model


In [253]:
pdPrediction=pd.DataFrame(columns=['_id','predict_severity','prob_severity'])

for  row_dict in dfNewData.to_dict(orient="records"):
      incident_id=row_dict['id']
      print(f"{incident_id} - {row_dict['severity']}({row_dict['severity_name']})") 
      for key_removed in unUsedColtoPredict:
       row_dict.pop(key_removed)
      # print(row_dict)  

      input_dict = {name: tf.convert_to_tensor([value]) for name, value in row_dict.items()}


      predictionResult = model.predict(input_dict)
      result_str=','.join([ str(prob) for prob in predictionResult[0]])  
      print(result_str)   

      prob = tf.nn.softmax(predictionResult)
      prob_pct=(100 * prob)  
      _class = tf.argmax(predictionResult,-1).numpy()[0]
      
      dictPrediction={'_id':incident_id, 'predict_severity':_class,'prob_severity':result_str} 
      pdPrediction =pd.concat([pdPrediction,pd.DataFrame.from_dict([dictPrediction])] )

      print(f"{prob_pct} %   as {_class}")     
      print("======================================================================================")
            
dfPredictData=pd.merge(dfNewData,pdPrediction,how='inner',left_on='id',right_on='_id')
dfPredictData=dfPredictData.drop(columns=['_id'])
dfPredictData['predict_severity']=dfPredictData['predict_severity'].astype('int')
dfPredictData=dfPredictData[['id','prob_severity','predict_severity','severity']]
dfPredictData['prediction_item_date']= datetime.strptime(str_yesterday, '%Y-%m-%d')
dfPredictData['prediction_datetime']=prediction_datetime
dfPredictData['model_version']=model_version


2856 - 2(Major)
1/1 [==============================] - 0s 393ms/step
0.083347686,0.64004886,0.2758465,0.0007570568
[[20.500584 35.771606 24.852346 18.87546 ]] %   as 1
3171 - 2(Major)
1/1 [==============================] - 0s 54ms/step
0.012354736,0.06561591,0.83755696,0.08447233
[[18.47804  19.488884 42.173214 19.85986 ]] %   as 2
3161 - 2(Major)
1/1 [==============================] - 0s 56ms/step
0.010114209,0.19694075,0.74444056,0.048504435
[[18.766542 22.62153  39.110928 19.501001]] %   as 2
3168 - 2(Major)
1/1 [==============================] - 0s 57ms/step
0.0041307607,0.039464105,0.95431346,0.0020917132
[[17.793245 18.43318  46.016575 17.757002]] %   as 2
3182 - 2(Major)
1/1 [==============================] - 0s 54ms/step
0.025347969,0.3689998,0.49316248,0.11248972
[[19.61692  27.661573 31.318436 21.403067]] %   as 2
3176 - 2(Major)
1/1 [==============================] - 0s 67ms/step
0.021658009,0.07210011,0.88723063,0.019011173
[[18.43162  19.385199 43.800285 18.3829  ]] %   as

In [254]:
print(dfPredictData.info())
dfPredictData[['id','prediction_item_date','predict_severity','severity']]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   id                    37 non-null     object             
 1   prob_severity         37 non-null     object             
 2   predict_severity      37 non-null     int32              
 3   severity              37 non-null     int64              
 4   prediction_item_date  37 non-null     datetime64[ns]     
 5   prediction_datetime   37 non-null     datetime64[ns, UTC]
 6   model_version         37 non-null     object             
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), int32(1), int64(1), object(3)
memory usage: 2.0+ KB
None


,id,prediction_item_date,predict_severity,severity
0,2856,2023-07-19,1,2
1,3171,2023-07-19,2,2
2,3161,2023-07-19,2,2
3,3168,2023-07-19,2,2
4,3182,2023-07-19,2,2
5,3176,2023-07-19,2,2
6,3048,2023-07-19,2,2
7,3063,2023-07-19,2,2
8,3193,2023-07-19,2,2
9,3148,2023-07-19,2,2


In [256]:
if  showEvaluationReport:
    from sklearn.metrics import classification_report

    className=list(set().union(list(dfPredictData['severity'].unique()),list(dfPredictData['predict_severity'].unique())))

    y_true=list(dfPredictData['severity'])
    y_pred=list(dfPredictData['predict_severity'])
    print(classification_report(y_true,y_pred, labels=className))

              precision    recall  f1-score   support

           0       1.00      0.88      0.93         8
           1       0.50      0.21      0.30        14
           2       0.50      0.80      0.62        15

    accuracy                           0.59        37
   macro avg       0.67      0.63      0.62        37
weighted avg       0.61      0.59      0.56        37



# Save predictoin resutl to Bigquery

In [94]:
#https://cloud.google.com/bigquery/docs/samples/bigquery-create-table#bigquery_create_table-python

try:
    client.get_table(predictResult_table_id)  # Make an API request.
    print("Predict Result Table {} already exists.".format(predictResult_table_id))
except Exception as ex:
    schema = [
    bigquery.SchemaField("id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("prob_severity", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("predict_severity", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("severity", "INTEGER", mode="REQUIRED"),    
    bigquery.SchemaField("prediction_item_date", "DATETIME", mode="REQUIRED"),    
    bigquery.SchemaField("prediction_datetime", "DATETIME", mode="REQUIRED"), 
    bigquery.SchemaField("model_version",  "STRING", mode="REQUIRED")     
    ]

    table = bigquery.Table(predictResult_table_id,schema=schema)
    table.time_partitioning = bigquery.TimePartitioning(
    type_=bigquery.TimePartitioningType.DAY,field="prediction_item_date")
    
    table = client.create_table(table)  # Make an API request.
    
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

Created table smart-data-ml.SMartML.new_result_prediction_incident


In [201]:
# def loadDataFrameToBQ():
#     try:
#         job_config = bigquery.LoadJobConfig(
#             write_disposition="WRITE_APPEND",
#         )

#         job = client.load_table_from_dataframe(
#             dfPredictData, predictResult_table_id, job_config=job_config
#         )
#         job.result()  # Wait for the job to complete.
#         print("Total Prediction ML ", len(dfPredictData), "Imported bigquery successfully")

#     except BadRequest as e:
#         print("Bigquery Error\n")
#         for e in job.errors:
#             print('ERROR: {}'.format(e['message']))

# try:
#     loadDataFrameToBQ()
# except Exception as ex:
#     raise ex

In [ ]:
# return 'ok'

In [ ]:
# if __name__ == "__main__":
#  result=predict_incident_severity_by_tf(None)
#  print(result)
